<a href="https://colab.research.google.com/github/DuarteVn/RAG-Simples/blob/main/PLN_Pr%C3%A1tica_07_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processamento de Linguagem Natural e IA Generativa
**Prática Aula 07: Retrieval Augmented Generation**

- Implementação do pipeline de um sistema RAG Simples

### PARTE 1: Instalação das dependências necessárias

In [ ]:
# transformers → Biblioteca da Hugging Face com modelos pré-treinados
# torch → framework de deep learning para redes neurais
# faiss-cpu → Para armazenamento e busca vetorial
# gradio → Criar interfaces web interativas rapidamente
# sentence-transformers → embeddings de sentenças

!pip install transformers torch faiss-cpu gradio sentence-transformers --quiet

### PARTE 2: Importações e configuração básica

In [ ]:
import os
import pickle
import faiss
import numpy as np
import gradio as gr
import torch
from typing import List, Dict

from transformers import AutoTokenizer, T5ForConditionalGeneration
# AutoTokenizer: carrega automaticamente o tokenizador correto para o modelo
# T5ForConditionalGeneration: modelo T5 para tarefas de geração de texto

from sentence_transformers import SentenceTransformer
# embeddings para busca semântica

# Verificar se GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo em uso: {device}")

# Configuração dos modelos
EMBEDDING_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
LLM_MODEL = "google/flan-t5-base"

Dispositivo em uso: cpu


### PARTE 3: Carregamento dos modelos

In [ ]:
embedding_model = SentenceTransformer(EMBEDDING_MODEL)

tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
model = T5ForConditionalGeneration.from_pretrained(LLM_MODEL)

# Mover para GPU se disponível
if device.type == "cuda":
    model = model.to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### PARTE 4 - Cria uma base de dados de exemplo para demonstração

In [ ]:
BASE_DIR = "DB/"

def criar_base_exemplo():
    """
    Cria uma base de dados de exemplo para demonstração
    """
    print("Criando base de dados de exemplo...")

    # Dados de exemplo
    documentos_exemplo = [
        {
            "text": "Células-tronco são células com capacidade de autorenovação e diferenciação em vários tipos celulares. Podem ser embrionárias, adultas ou induzidas.",
            "source": "Manual de Biologia Celular"
        },
        {
            "text": "A dengue é uma doença viral transmitida pelo mosquito Aedes aegypti. Sintomas incluem febre alta, dor de cabeça e dores musculares.",
            "source": "Guia de Doenças Tropicais"
        },
        {
            "text": "Vacinas funcionam estimulando o sistema imunológico a reconhecer patógenos específicos sem causar a doença.",
            "source": "Fundamentos da Imunologia"
        },
        {
            "text": "Epidemiologia estuda a distribuição e fatores determinantes de doenças em populações específicas.",
            "source": "Princípios de Epidemiologia"
        }
    ]

    try:
        # Criar diretório
        caminho_exemplo = os.path.join(BASE_DIR, "base_biologia")
        os.makedirs(caminho_exemplo, exist_ok=True)

        # Gerar embeddings
        embeddings = []
        for doc in documentos_exemplo:
            embedding = criar_embedding(doc["text"])
            embeddings.append(embedding)

        # Criar índice FAISS
        embeddings_array = np.array(embeddings).astype('float32')
        dimensao = embeddings_array.shape[1]
        indice = faiss.IndexFlatL2(dimensao)
        indice.add(embeddings_array)

        # Salvar arquivos
        faiss.write_index(indice, os.path.join(caminho_exemplo, "faiss_index.index"))

        with open(os.path.join(caminho_exemplo, "chunk_metadata.pkl"), "wb") as f:
            pickle.dump(documentos_exemplo, f)

        print("Base de exemplo criada com sucesso!")

    except Exception as e:
        print(f"Erro ao criar base de exemplo: {e}")

criar_base_exemplo()

Criando base de dados de exemplo...
Erro ao criar base de exemplo: name 'criar_embedding' is not defined


### PARTE 5: Funções principais

In [ ]:
def criar_embedding(texto: str) -> List[float]:
    """
    Converte texto em vetor numérico (embedding)
    """
    embedding = embedding_model.encode(texto, convert_to_tensor=False)
    return embedding.tolist()

def buscar_documentos_relevantes(pergunta: str, indice_faiss, lista_documentos, k=3):
    """
    Busca os documentos mais similares à pergunta usando BD Vetorial FAISS
    """
    try:
        print(f"Buscando documentos para: {pergunta[:50]}...")

        # Verificar se há documentos
        if not lista_documentos:
            print("Erro: Lista de documentos vazia")
            return []

        # Criar embedding da pergunta
        embedding_pergunta = criar_embedding(pergunta)
        if not embedding_pergunta:
            print("Erro: Não foi possível criar embedding da pergunta")
            return []

        # Converter para formato FAISS
        vetor_pergunta = np.array(embedding_pergunta).astype('float32').reshape(1, -1)
        print(f"Vetor criado com dimensão: {vetor_pergunta.shape}")

        # Ajustar k se necessário
        k_ajustado = min(k, len(lista_documentos), indice_faiss.ntotal)
        print(f"Buscando {k_ajustado} documentos de {len(lista_documentos)} disponíveis")

        # Buscar documentos similares
        distancias, indices = indice_faiss.search(vetor_pergunta, k_ajustado)

        print(f"Distâncias encontradas: {distancias[0]}")
        print(f"Índices encontrados: {indices[0]}")

        # Verificar se encontrou resultados válidos
        if len(indices[0]) == 0:
            print("Nenhum índice retornado pela busca")
            return lista_documentos[:k_ajustado]  # Retorna primeiros documentos como fallback

        # Retornar documentos encontrados
        documentos_encontrados = []
        for i in indices[0]:
            if 0 <= i < len(lista_documentos):
                documentos_encontrados.append(lista_documentos[i])
                print(f"Documento {i}: {lista_documentos[i]['text'][:100]}...")

        if not documentos_encontrados:
            print("Nenhum documento válido encontrado, usando fallback")
            return lista_documentos[:k_ajustado]

        print(f"Retornando {len(documentos_encontrados)} documentos")
        return documentos_encontrados

    except Exception as e:
        print(f"Erro na busca: {e}")
        # Fallback: retorna primeiros documentos
        return lista_documentos[:min(k, len(lista_documentos))]


def gerar_resposta(pergunta: str, documentos_contexto: List[Dict]) -> str:
    """
    Gera resposta com base no contexto fornecido
    """
    # Preparar o contexto
    contexto = "\n\n".join([doc["text"] for doc in documentos_contexto])

    # Criar prompt estruturado
    prompt = f"""Com base nas informações fornecidas, responda à pergunta em português brasileiro.

INFORMAÇÕES:
{contexto}

PERGUNTA: {pergunta}

RESPOSTA:"""

    try:
        # Tokenizar entrada
        inputs = tokenizer.encode(
            prompt,
            return_tensors="pt",
            max_length=512,
            truncation=True
        )

        # Mover para GPU se necessário
        if device.type == "cuda":
            inputs = inputs.to(device)

        # Gerar resposta
        with torch.no_grad():
            outputs = model.generate(
                inputs,
                max_new_tokens=200,
                temperature=0.3,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id
            )

        # Decodificar resposta
        resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Limpar resposta (remover prompt)
        if "RESPOSTA:" in resposta:
            resposta = resposta.split("RESPOSTA:")[-1].strip()

        # Adicionar fontes
        fontes = [f"- {doc['source']}" for doc in documentos_contexto]
        if fontes:
            resposta += "\n\nFontes:\n" + "\n".join(fontes)

        return resposta

    except Exception as e:
        print(f"Erro na geração: {e}")
        return f"Não foi possível gerar uma resposta adequada. Erro: {str(e)}"

### PARTE 6: Funções de gerenciamento de bases de dados

In [ ]:
def listar_bases_disponiveis():
    """
    Lista todas as bases FAISS disponíveis no diretório
    """
    bases = []
    try:
        for item in os.listdir(BASE_DIR):
            caminho_base = os.path.join(BASE_DIR, item)
            if os.path.isdir(caminho_base):
                arquivo_indice = os.path.join(caminho_base, "faiss_index.index")
                arquivo_metadados = os.path.join(caminho_base, "chunk_metadata.pkl")

                if os.path.exists(arquivo_indice) and os.path.exists(arquivo_metadados):
                    bases.append(item)
    except:
        pass

    return bases

def carregar_base_faiss(nome_base):
    """
    Carrega uma base FAISS específica
    """
    if not nome_base:
        raise ValueError("Nome da base não fornecido")

    caminho_indice = os.path.join(BASE_DIR, nome_base, "faiss_index.index")
    caminho_metadados = os.path.join(BASE_DIR, nome_base, "chunk_metadata.pkl")

    # Verificar se existem arquivos
    if not os.path.exists(caminho_indice):
        raise FileNotFoundError(f"Índice não encontrado: {caminho_indice}")

    if not os.path.exists(caminho_metadados):
        raise FileNotFoundError(f"Metadados não encontrados: {caminho_metadados}")

    # Carregar arquivos
    indice = faiss.read_index(caminho_indice)

    with open(caminho_metadados, "rb") as f:
        documentos = pickle.load(f)

    print(f"Base carregada: {nome_base} ({len(documentos)} documentos)")
    return indice, documentos

### PARTE 7: Função principal do chat

In [ ]:
def chat_principal(mensagem, historico, escolha_base, escolha_modelo):
    """
    Função principal que processa as mensagens do chat
    """
    try:
        # Validações básicas
        if not mensagem.strip():
            return "Por favor, digite uma pergunta."

        if not escolha_base:
            return "Por favor, selecione uma base de conhecimento primeiro."

        # Carregar base de dados
        indice, documentos = carregar_base_faiss(escolha_base)

        # Buscar documentos relevantes
        documentos_relevantes = buscar_documentos_relevantes(mensagem, indice, documentos)

        if not documentos_relevantes:
            return "Não foram encontradas informações relevantes na base selecionada."

        # Gerar resposta
        resposta = gerar_resposta(mensagem, documentos_relevantes)

        return resposta

    except Exception as e:
        return f"Erro ao processar pergunta: {str(e)}"



### PARTE 8: Interface do ChatBot com Gradio

In [ ]:
def criar_interface():
    """
    Cria a interface web usando Gradio
    """
    # Obter bases disponíveis
    bases_disponiveis = listar_bases_disponiveis()

    # Se não há bases, criar uma de exemplo
    if not bases_disponiveis:
        print("Nenhuma base encontrada. Criando base de exemplo...")
        if criar_base_exemplo():
            bases_disponiveis = listar_bases_disponiveis()

    with gr.Blocks(title="PrimeiroRAG Simplificado") as interface:

        # Título
        gr.Markdown("# PrimeiroRAG - Sistema RAG Local")
        gr.Markdown("Sistema de perguntas e respostas usando modelos de IA locais")

        with gr.Row():
            # Coluna de configurações
            with gr.Column(scale=1):
                gr.Markdown("### Configurações")

                # Seleção da base
                dropdown_base = gr.Dropdown(
                    choices=bases_disponiveis,
                    value=bases_disponiveis[0] if bases_disponiveis else None,
                    label="Base de Conhecimento",
                    info="Selecione a base de dados"
                )

                # Seleção do modelo (apenas visual, só temos um)
                dropdown_modelo = gr.Dropdown(
                    choices=["FLAN-T5 Base"],
                    value="FLAN-T5 Base",
                    label="Modelo de IA",
                    info="Modelo de linguagem local"
                )

                # Botão para atualizar bases
                botao_atualizar = gr.Button("Atualizar Bases")

            # Coluna principal do chat
            with gr.Column(scale=3):
                gr.Markdown("### Chat")

                # Interface do chat
                chat_interface = gr.ChatInterface(
                    fn=chat_principal,
                    description="Faça perguntas sobre o conteúdo da base de conhecimento selecionada.",
                    type="messages",
                    additional_inputs=[dropdown_base, dropdown_modelo],
                    submit_btn="Enviar"
                )

        # Função para atualizar lista de bases
        def atualizar_bases():
            novas_bases = listar_bases_disponiveis()
            return gr.Dropdown(choices=novas_bases)

        # Conectar botão de atualizar
        botao_atualizar.click(atualizar_bases, outputs=dropdown_base)

    return interface


### PARTE 9: Executar aplicação


def executar_PrimeiroRAG():
    """
    Executa o sistema PrimeiroRAG
    """
    print("Iniciando PrimeiroRAG...")

    # Verificar se há bases disponíveis
    bases = listar_bases_disponiveis()
    print(f"Bases disponíveis: {len(bases)}")

    if not bases:
        print("Criando base de exemplo...")
        criar_base_exemplo()

    # Criar e lançar interface
    interface = criar_interface()

    print("Lançando interface...")
    interface.launch(
        share=True,
        debug=True
    )

# Executar o sistema
if __name__ == "__main__":
    executar_PrimeiroRAG()

Iniciando PrimeiroRAG...
Bases disponíveis: 2
Lançando interface...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://edeaddafb4481ee190.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Base carregada: base_ciencia_dados (5 documentos)
Buscando documentos para: Quais são as etapas de um pipeline?...
Vetor criado com dimensão: (1, 384)
Buscando 3 documentos de 5 disponíveis
Distâncias encontradas: [18.22726 31.43446 35.32258]
Índices encontrados: [1 4 3]
Documento 1: Um pipeline de Ciência de Dados tem várias etapas: coleta dos dados, limpeza, análise exploratória (...
Documento 4: Deploy é colocar o modelo para funcionar com dados reais. MLOps é o conjunto de boas práticas para m...
Documento 3: As métricas servem para medir a qualidade dos modelos. Na classificação, usamos: precisão, recall, F...
Retornando 3 documentos
Base carregada: base_ciencia_dados (5 documentos)
Buscando documentos para: O que é deploy?...
Vetor criado com dimensão: (1, 384)
Buscando 3 documentos de 5 disponíveis
Distâncias encontradas: [21.565866 31.578342 35.92127 ]
Índices encontrados: [4 1 0]
Documento 4: Deploy é colocar o modelo para funcionar com dados reais. MLOps é o conjunto de boas 

### Exercício

- Inclua mais uma base de conhecimento, chamada "base_ciência_dados" com textos sobre ciência de dados
- Ao selecionar a nova base na caixa de seleção "Base de Conhecimento - Selecione a base de dados" da interface, o RAG deve responder perguntas sobre a base selecionada.

In [ ]:
BASE_DIR = "DB/"

def criar_base_ciencia_dados():
    """
    Cria uma base de dados
    """
    print("Criando base de dados sobre Ciência de Dados...")

    # Dados de exemplo
    documentos = [
        {
            "text":
                "Ciência de Dados é a área que usa estatística, programação e conhecimento de negócio para analisar dados e gerar insights. "
                "Ela ajuda empresas a entender padrões, prever situações e tomar decisões melhores."
            ,
            "source": "Introdução à Ciência de Dados"
        },
        {
            "text":
                "Um pipeline de Ciência de Dados tem várias etapas: "
                "coleta dos dados, limpeza, análise exploratória (EDA), criação de novas variáveis, escolha do modelo, avaliação e deployment. "
                "Cada etapa ajuda a transformar dados brutos em soluções úteis."
            ,
            "source": "Pipeline de Ciência de Dados"
        },
        {
            "text":
                "Embeddings são formas de transformar textos em números. "
                "Com isso, o computador consegue comparar frases, buscar respostas parecidas e melhorar a precisão de modelos como o RAG. "
                "Modelos como BERT e MiniLM são usados para gerar esses vetores numéricos."
            ,
            "source": "Embeddings e Busca Semântica"
        },
        {
            "text":
                "As métricas servem para medir a qualidade dos modelos. "
                "Na classificação, usamos: precisão, recall, F1-score e AUC-ROC. "
                "Na regressão, usamos: MAE, RMSE e R². "
                "No ranking, usamos: MAP e NDCG."
            ,
            "source": "Métricas em Modelos de Machine Learning"
        },
        {
            "text":
                "Deploy é colocar o modelo para funcionar com dados reais. "
                "MLOps é o conjunto de boas práticas para manter esse modelo estável. "
                "Isso inclui: testes, monitoramento, alertas e re-treinamento quando o modelo começa a errar."
            ,
            "source": "Boas Práticas de Deploy e MLOps"
        }
    ]



    try:
        # Criar diretório
        caminho_exemplo = os.path.join(BASE_DIR, "base_ciencia_dados")
        os.makedirs(caminho_exemplo, exist_ok=True)

        # Gerar embeddings
        embeddings = []
        for doc in documentos:
            embedding = criar_embedding(doc["text"])
            embeddings.append(embedding)

        # Criar índice FAISS
        embeddings_array = np.array(embeddings).astype('float32')
        dimensao = embeddings_array.shape[1]
        indice = faiss.IndexFlatL2(dimensao)
        indice.add(embeddings_array)

        # Salvar arquivos
        faiss.write_index(indice, os.path.join(caminho_exemplo, "faiss_index.index"))

        with open(os.path.join(caminho_exemplo, "chunk_metadata.pkl"), "wb") as f:
            pickle.dump(documentos, f)

        print("Base de exemplo criada com sucesso!")

    except Exception as e:
        print(f"Erro ao criar base de exemplo: {e}")

criar_base_ciencia_dados()